In [3]:
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.linear_model import LogisticRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats.stats import pearsonr
from scipy.stats import norm

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import (
    RandomForestClassifier,
    ExtraTreesClassifier,
    AdaBoostClassifier,
    GradientBoostingClassifier,
)
from sklearn.model_selection import cross_validate

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score

## Loading Data

In [4]:
with open('X_train.npy', 'rb') as f:
    X_train = np.load(f)
with open('X_test.npy', 'rb') as f:
    X_test = np.load(f)
with open('y_train.npy', 'rb') as f:
    y_train = np.load(f)
with open('y_test.npy', 'rb') as f:
    y_test = np.load(f)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(13972, 28) (4069, 28) (13972,) (4069,)


In [5]:
with open('X_train2.npy', 'rb') as f:
    X_train2 = np.load(f)
with open('X_test2.npy', 'rb') as f:
    X_test2 = np.load(f)
# with open('X_train3.npy', 'rb') as f:
#     X_train3 = np.load(f)
# with open('X_test3.npy', 'rb') as f:
#     X_test3 = np.load(f)

In [6]:
columns = ['Administrative',
 'Administrative_Duration',
 'Informational',
 'Informational_Duration',
 'ProductRelated',
 'ProductRelated_Duration',
 'BounceRates',
 'ExitRates',
 'PageValues',
 'SpecialDay',
 'Month_Aug',
 'Month_Dec',
 'Month_Feb',
 'Month_Jul',
 'Month_June',
 'Month_Mar',
 'Month_May',
 'Month_Nov',
 'Month_Oct',
 'Month_Sep',
 'VisitorType_New_Visitor',
 'VisitorType_Other',
 'VisitorType_Returning_Visitor',
 'Weekend_True',
 'OperatingSystems',
 'Browser',
 'Region',
 'TrafficType']

feature_names = columns

In [7]:
X_train = pd.DataFrame(X_train, columns=feature_names)
X_test = pd.DataFrame(X_test, columns=feature_names)

In [8]:
X_DF = pd.concat([X_train,X_test])
y = np.concatenate((y_train,y_test))

## Tuning Hyperparameters

In [9]:
model = ExtraTreesClassifier()

In [10]:
param_dist = {
    "n_estimators": list(np.linspace(200, 1600, 8, dtype=int)),
    "max_features": ["auto", "sqrt"],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_depth": [4, 6, 8, None]
}

In [11]:
def get_best_params(model,param_dist,X,y):
    model_rand = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=100, cv=3, verbose=2, random_state=0, n_jobs=-1)
    model_rand.fit(X,y)
    best_params = model_rand.best_params_
    return best_params

In [12]:
best_params = get_best_params(model,param_dist,X_train,y_train)
print(best_params)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
{'n_estimators': 800, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': None}


## Fitting the Model

In [13]:
model_best = ExtraTreesClassifier(**best_params)
model_best.fit(X_train,y_train) 

ExtraTreesClassifier(n_estimators=800)

In [14]:
score = cross_val_score(model_best, X_train, y_train)
print(np.round(score.mean(), 4))

0.9201


In [15]:
def get_performance(model,X,y):
    y_pred = model.predict(X)
    target_names = ['0: No Revenue', '1: Revenue']
    report = classification_report(y, y_pred, target_names=target_names)
    
    report_file = classification_report(y, y_pred, target_names=target_names, output_dict=True)
    
    print(report_file)
    print('\n')
    
    report_file = pd.DataFrame(report_file).transpose()
    report_file.to_csv(f"{str(model)}_report1.csv",index=False)
    
    print(report)
    print("Confusion Matrix:\n",confusion_matrix(y,y_pred))
    print("Accuracy: ",accuracy_score(y,y_pred))
    print("ROC-AUC: ",roc_auc_score(y,model.predict_proba(X)[:,1]))

In [16]:
get_performance(model_best,X_test,y_test)

{'0: No Revenue': {'precision': 0.9285506825442927, 'recall': 0.930442374854482, 'f1-score': 0.9294955662160198, 'support': 3436}, '1: Revenue': {'precision': 0.6182108626198083, 'recall': 0.6113744075829384, 'f1-score': 0.6147736298649722, 'support': 633}, 'accuracy': 0.8808060948636028, 'macro avg': {'precision': 0.7733807725820505, 'recall': 0.7709083912187102, 'f1-score': 0.772134598040496, 'support': 4069}, 'weighted avg': {'precision': 0.8802722096978444, 'recall': 0.8808060948636028, 'f1-score': 0.8805353829498086, 'support': 4069}}


               precision    recall  f1-score   support

0: No Revenue       0.93      0.93      0.93      3436
   1: Revenue       0.62      0.61      0.61       633

     accuracy                           0.88      4069
    macro avg       0.77      0.77      0.77      4069
 weighted avg       0.88      0.88      0.88      4069

Confusion Matrix:
 [[3197  239]
 [ 246  387]]
Accuracy:  0.8808060948636028
ROC-AUC:  0.9108103125166669


## Model with Feature Engineering

In [17]:
best_params = get_best_params(model,param_dist,X_train2,y_train)
print(best_params)

model2 = ExtraTreesClassifier(**best_params)
model2.fit(X_train2, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
{'n_estimators': 400, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': None}


ExtraTreesClassifier(n_estimators=400)

In [18]:
get_performance(model2,X_test2,y_test)

{'0: No Revenue': {'precision': 0.9459459459459459, 'recall': 0.9167636786961583, 'f1-score': 0.9311262193319538, 'support': 3436}, '1: Revenue': {'precision': 0.6129905277401895, 'recall': 0.7156398104265402, 'f1-score': 0.6603498542274052, 'support': 633}, 'accuracy': 0.8854755468173998, 'macro avg': {'precision': 0.7794682368430677, 'recall': 0.8162017445613492, 'f1-score': 0.7957380367796796, 'support': 4069}, 'weighted avg': {'precision': 0.8941492441213591, 'recall': 0.8854755468173998, 'f1-score': 0.8890024938192531, 'support': 4069}}


               precision    recall  f1-score   support

0: No Revenue       0.95      0.92      0.93      3436
   1: Revenue       0.61      0.72      0.66       633

     accuracy                           0.89      4069
    macro avg       0.78      0.82      0.80      4069
 weighted avg       0.89      0.89      0.89      4069

Confusion Matrix:
 [[3150  286]
 [ 180  453]]
Accuracy:  0.8854755468173998
ROC-AUC:  0.9266345377537715


## Model with Feature Selection and Feature Engineering

In [19]:
with open('X_alt_train.npy', 'rb') as f:
     X_alt_train = np.load(f)
with open('X_alt_test.npy', 'rb') as f:
     X_alt_test = np.load(f)
        
to_keep = [10, 11, 21, 19, 13, 9, 23, 3, 22, 0, 20, 14, 2, 15, 1, 27, 16, 4, 25, 18]

def get_X(to_keep,X_train,X_test,X_alt_train,X_alt_test):
    X_train_ss = X_train.iloc[:,to_keep]
    X_test_ss = X_test.iloc[:,to_keep]
    
    X_train3 = np.hstack((X_train_ss.to_numpy(),X_alt_train))
    X_test3 = np.hstack((X_test_ss.to_numpy(),X_alt_test))
    
    return X_train3, X_test3

X_train3,X_test3 = get_X(to_keep,X_train,X_test,X_alt_train,X_alt_test)

np.save("X_trainET.npy",X_train3)
np.save("X_testET.npy",X_test3)

In [20]:
best_params = get_best_params(model,param_dist,X_train3,y_train)
print(best_params)

model3 = ExtraTreesClassifier(**best_params)
model3.fit(X_train3, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
{'n_estimators': 400, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': None}


ExtraTreesClassifier(min_samples_split=5, n_estimators=400)

In [21]:
get_performance(model3,X_test3,y_test)

{'0: No Revenue': {'precision': 0.9442771084337349, 'recall': 0.9123981373690337, 'f1-score': 0.9280639431616341, 'support': 3436}, '1: Revenue': {'precision': 0.5981308411214953, 'recall': 0.707740916271722, 'f1-score': 0.6483357452966715, 'support': 633}, 'accuracy': 0.8805603342344557, 'macro avg': {'precision': 0.7712039747776152, 'recall': 0.8100695268203779, 'f1-score': 0.7881998442291528, 'support': 4069}, 'weighted avg': {'precision': 0.8904283526685229, 'recall': 0.8805603342344557, 'f1-score': 0.8845476125525111, 'support': 4069}}


               precision    recall  f1-score   support

0: No Revenue       0.94      0.91      0.93      3436
   1: Revenue       0.60      0.71      0.65       633

     accuracy                           0.88      4069
    macro avg       0.77      0.81      0.79      4069
 weighted avg       0.89      0.88      0.88      4069

Confusion Matrix:
 [[3135  301]
 [ 185  448]]
Accuracy:  0.8805603342344557
ROC-AUC:  0.9227540565741051
